In [7]:
import asyncio
from playwright.async_api import async_playwright

async def crawl_pasgo_by_page(keyword, max_pages=5):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        all_results = []

        for page_num in range(1, max_pages + 1):
            url = f"https://pasgo.vn/tim-kiem?search={keyword}&page={page_num}"
            print(f"🔎 Crawling page {page_num}: {url}")
            await page.goto(url)
            try:
                await page.wait_for_selector("div.item-child-info a", timeout=8000)
            except:
                print(f"⛔ Không tìm thấy dữ liệu ở trang {page_num}")
                continue

            items = await page.query_selector_all("div.item-child-info a")
            for item in items:
                name = await item.inner_text()
                link = await item.get_attribute("href")
                full_link = f"https://pasgo.vn{link}"
                print(f"{name.strip()} - {full_link}")
                all_results.append((name.strip(), full_link))

        await browser.close()
        return all_results

# Ví dụ: Crawl 5 trang kết quả cho từ khoá "lẩu"
results = await crawl_pasgo_by_page("lẩu", max_pages=5)


🔎 Crawling page 1: https://pasgo.vn/tim-kiem?search=lẩu&page=1
Lộc-ally - Cát Linh

Tầng 2, Khách Sạn Grand Mercure Hanoi, Số 9 Cát Linh, P. Quốc Tử Giám,Q. Ba Đình

Đặt bàn giữ chỗ
Buffet Món Việt, Gọi món Á - Âu - https://pasgo.vnhttps://pasgo.vn/nha-hang/loc-ally-restaurant-cat-linh-5490
Đặt chỗ - https://pasgo.vnhttps://pasgo.vn/dat-cho-ngay/5490?returnUrl=/tim-kiem?search=l%E1%BA%A9u&page=1
GoGi House - Giang Văn Minh

Số 14 Giang Văn Minh, P. Kim Mã, Q. Ba Đình

Đặt bàn giữ chỗ
Gọi món, Buffet Nướng Lẩu Hàn Quốc - https://pasgo.vnhttps://pasgo.vn/nha-hang/gogi-house-giang-van-minh-5575
Đặt chỗ - https://pasgo.vnhttps://pasgo.vn/dat-cho-ngay/5575?returnUrl=/tim-kiem?search=l%E1%BA%A9u&page=1
Lẩu Nấm Ashima - Giang Văn Minh

Số 60 Giang Văn Minh, P. Đội Cấn, Q. Ba Đình

Ưu đãi hấp dẫn
Gọi món Á, Chuyên Lẩu Nấm - https://pasgo.vnhttps://pasgo.vn/nha-hang/nha-hang-lau-nam-ashima-giang-van-minh-963
Đặt chỗ - https://pasgo.vnhttps://pasgo.vn/dat-cho-ngay/963?returnUrl=/tim-kiem?sea

In [10]:
import asyncio
from playwright.async_api import async_playwright

async def crawl_pasgo_by_city(keyword, city_id=2, max_pages=3):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await p.chromium.launch_persistent_context(
            user_data_dir="/tmp/playwright",  # cache tạm
            headless=True
        )

        # Gán cityId ngay từ đầu bằng add_init_script
        await context.add_init_script(f"""
            localStorage.setItem('cityId', '{city_id}');
        """)

        page = await context.new_page()
        all_results = []

        for page_num in range(1, max_pages + 1):
            url = f"https://pasgo.vn/tim-kiem?search={keyword}&page={page_num}"
            print(f"🔎 Crawling page {page_num}: {url}")
            await page.goto(url, wait_until="load")

            try:
                await page.wait_for_selector("div.item-child-info a", timeout=8000)
            except:
                print(f"⛔ Trang {page_num} không có kết quả")
                continue

            items = await page.query_selector_all("div.item-child-info a")
            for item in items:
                name = await item.inner_text()
                link = await item.get_attribute("href")
                full_url = f"https://pasgo.vn{link}"
                print(f"📍 {name.strip()} - {full_url}")
                all_results.append((name.strip(), full_url))

        await browser.close()
        return all_results

# Hồ Chí Minh = city_id 2
results = await crawl_pasgo_by_city("lẩu", city_id=2, max_pages=2)


🔎 Crawling page 1: https://pasgo.vn/tim-kiem?search=lẩu&page=1
📍 Lộc-ally - Cát Linh

Tầng 2, Khách Sạn Grand Mercure Hanoi, Số 9 Cát Linh, P. Quốc Tử Giám,Q. Ba Đình

Đặt bàn giữ chỗ
Buffet Món Việt, Gọi món Á - Âu - https://pasgo.vnhttps://pasgo.vn/nha-hang/loc-ally-restaurant-cat-linh-5490
📍 Đặt chỗ - https://pasgo.vnhttps://pasgo.vn/dat-cho-ngay/5490?returnUrl=/tim-kiem?search=l%E1%BA%A9u&page=1
📍 GoGi House - Giang Văn Minh

Số 14 Giang Văn Minh, P. Kim Mã, Q. Ba Đình

Đặt bàn giữ chỗ
Gọi món, Buffet Nướng Lẩu Hàn Quốc - https://pasgo.vnhttps://pasgo.vn/nha-hang/gogi-house-giang-van-minh-5575
📍 Đặt chỗ - https://pasgo.vnhttps://pasgo.vn/dat-cho-ngay/5575?returnUrl=/tim-kiem?search=l%E1%BA%A9u&page=1
📍 Lẩu Nấm Ashima - Giang Văn Minh

Số 60 Giang Văn Minh, P. Đội Cấn, Q. Ba Đình

Ưu đãi hấp dẫn
Gọi món Á, Chuyên Lẩu Nấm - https://pasgo.vnhttps://pasgo.vn/nha-hang/nha-hang-lau-nam-ashima-giang-van-minh-963
📍 Đặt chỗ - https://pasgo.vnhttps://pasgo.vn/dat-cho-ngay/963?returnUrl=/

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from playwright.async_api import async_playwright

# ARTICLE_EXTRACTORS = {
#     "NH-DEXUAT": extract_recommendation,
#     "NH-TOMTAT": extract_summary,
#     "NH-BANGGIA": extract_price,
#     "NH-QUYDINH": extract_rules,
#     "NH-DEXE": extract_parking,
#     "NH-TIENICH": extract_facilities,
#     "NH-ANH": extract_photos,
#     "NH-CHIDUONG": extract_map,
#     "NH-GIOHOATDONG": extract_opening_hours,
#     "NH-CHITIET": extract_detail_info,
#     "binh-luan": extract_reviews,
#     "detail-related": extract_related,
#     "detail-viewed": extract_viewed
# }
# ARTICLE_EXTRACTORS = {
#     "NH-TOMTAT": extract_summary,
# }
# async def extract_summary(page):
#     summary_element = await page.query_selector("div.nh-tomtat")
#     if summary_element:
#         return await summary_element.inner_text()
#     return "Không tìm thấy nội dung tóm tắt"
async def extract_summary(article, all_results):
    result = {}
    result['description'] = ''
    titles = await article.query_selector_all(".txt-title")
    for title in titles:
        title_text = (await title.inner_text()).strip().replace(":", "").upper()

        if "MÓN ĐẶC SẮC" in title_text:
            span = await article.query_selector("span")
            value = (await span.inner_text()).strip() if span else ""
            result['cuisines'] = value
        elif "ĐIỂM ĐẶC TRƯNG" in title_text:
            ps_texts = []
            sibling = await title.evaluate_handle("el => el.nextElementSibling")
            while sibling:
                is_null = await sibling.evaluate("el => el === null")
                if is_null:
                    break
                tag_name = await sibling.evaluate("el => el.tagName")
                if tag_name == "DIV":
                    class_name = await sibling.get_attribute("class")
                    if class_name == "txt-title":
                        break  # gặp title tiếp theo → dừng
                if tag_name == "P":
                    ps_texts.append((await sibling.inner_text()).strip())
                sibling = await sibling.evaluate_handle("el => el.nextElementSibling")
            value = "\n".join(ps_texts)
            result['description'] += value
        elif "THÔNG TIN THÊM" in title_text:
            pass    
        else:
            sibling = await title.evaluate_handle("el => el.nextElementSibling")
            value = ""
            if sibling:
                is_null = await sibling.evaluate("el => el === null")
                if not is_null:
                    tag_name = await sibling.evaluate("el => el.tagName")
                    if tag_name == "DIV" and await sibling.get_attribute("class") == "text-description":
                        value = (await sibling.inner_text()).strip()
                    elif tag_name == "P":
                        value = (await sibling.inner_text()).strip()
            result['description'] += value


    return result


 

async def get_detail_data(page, detail_link):
    await page.goto(detail_link)
    await page.wait_for_load_state("domcontentloaded")

    articles = await page.query_selector_all("article")
    article_data = {}

    for article in articles:
        article_id = await article.get_attribute("id")
        if article_id == "NH-TOMTAT":
            summary = await extract_summary(article)
            print(summary)
        print(article_id)



async def crawl_pasgo_by_page(keyword, max_pages=5):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        all_results = []

        for page_num in range(1, max_pages + 1):
            url = f"https://pasgo.vn/ha-noi/nha-hang?page={page_num}"
            print(f"🔎 Crawling page {page_num}: {url}")
            await page.goto(url)
            try:
                await page.wait_for_selector("div.wapitem a", timeout=8000)
            except:
                print(f"⛔ Không tìm thấy dữ liệu ở trang {page_num}")
                continue

            items = await page.query_selector_all("div.wapitem")
            for item in items:
                main = await item.query_selector("div.waptop-main")
                desc = await item.query_selector("div.waptop-desc")
                
                link = await item.query_selector("a.waptop")
                link = await link.get_attribute("href")
                full_link = f"https://pasgo.vn{link}" if link.startswith("/") else link
                detail_page = await browser.new_page()
                await get_detail_data(detail_page, full_link)
                await detail_page.close()

                img = await main.query_selector("a.waptop img")
                img_url = await img.get_attribute("src") if img else None
                
                name = await main.query_selector("div.wapfooter h3")
                name = await name.inner_text() if name else "Không có tên"

                address = await main.query_selector("p")
                address = await address.inner_text() if address else "Không có địa chỉ"


                sale = await desc.query_selector("div.wapsale")
                sale = await sale.inner_text() if sale else "Không có sale"

                tag = await desc.query_selector("div.waptag")
                tag = await tag.inner_text() if tag else "Không có tag"

                # print(f"{name.strip()} - {address.strip()} - {full_link} - {sale.strip()} - {tag.strip()} - {img_url}")
                all_results.append({
                    "name": name.strip(),
                    "address": address.strip(),
                    "link": full_link
                })

        await browser.close()
        return all_results

# Chạy thử
if __name__ == "__main__":
    asyncio.run(crawl_pasgo_by_page("lẩu", max_pages=5))


🔎 Crawling page 1: https://pasgo.vn/ha-noi/nha-hang?page=1
info-booth
None
NH-DEXUAT
{'summary': {'PHÙ HỢP': 'Cặp đôi, gia đình, khách văn phòng, sinh nhật...', 'MÓN ĐẶC SẮC': 'Thăn bò ngoại Hokubee, Thăn vai bò đặc biệt, Thăn vai bò Mỹ, Mỳ ý thịt băm, Mỳ thịt nguội sốt kem, Kem cháy cùng bánh quy giòn...', 'KHÔNG GIAN': '- Ấm cúng, nhẹ nhàng\xa0\n\n- Sức chứa: 40 Khách', 'CHỖ ĐỂ XE': '- Xe ô tô: Để xe trước cửa nhà hàng (Miễn phí)\n\n- Xe máy: Để xe trước cửa nhà hàng (Miễn phí)', 'ĐIỂM ĐẶC TRƯNG': '- Tại Botanica, các đầu bếp rất có tâm với nghề, và đã từng làm cho các khách sạn 5 sao tại Việt Nam và quốc tế, rất am hiểu về chế biến món ăn mang phong cách Âu để đảm bảo các món ăn chất lượng luôn được trọn vị.\n- Bánh mỳ home-made riêng của hệ thống: đặc ruột thơm ngậy bơ luôn là món ưa thích suốt gần 10 năm nay khi khách hàng thưởng thức cùng với Bít tết và các món nướng khác.\n- Nhà hàng luôn sử dụng những nguyên liệu tươi sạch để chế biến các món ăn.', 'THÔNG 

KeyboardInterrupt: 